In [ ]:
## API:: https://jsonplaceholder.typicode.com/posts?userId=2 ##
## Snowflake Account URL: https://ri36021.ap-south-1.aws.snowflakecomputing.com ##

In [1]:
type(f'{2}')

str

In [ ]:
!pip install snowflake-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 9.0 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import snowflake.connector
import requests
import json
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
account = 'ri36021.ap-south-1'
warehouse = 'COMPUTE_WH'
database = 'SAMPLE_DB'
schema ='SAMPLE_SCHEMA'
table = 'USER_POSTS_RAW_DATA'

In [ ]:
def snowflake_connector():
  # Connect to Snowflake
  conn = snowflake.connector.connect(
      user=userdata.get('snowflake_user'),
      password=userdata.get('snowflake_password'),
      account=account,
      warehouse=warehouse,
      database=database,
      schema=schema
  )
  return conn

def query_to_snowflake(query, mod='read'):
  try:
    conn = snowflake_connector()
    cur = conn.cursor()
    cur.execute(""" ALTER SESSION SET TIMEZONE = 'Asia/Kolkata';""")
    # cur.execute(""" ALTER SESSION SET TIMESTAMP_NTZ_OUTPUT_FORMAT = 'YYYY-MM-DD HH12:MI:SS AM';""")

    if mod.lower() == 'read':
      fecth_result = cur.execute(query)

    if mod.lower() == 'insert':
      fecth_result = cur.execute(query)
      return 'insert operation completed sucessfully'
      # Commit the transaction
      # cur.execute("COMMIT")

    return fecth_result.fetchall()

  except Exception as e:
    print(f'ERROR while invoking query to snowflake:: {e}')
  finally:

    # Close connections
    cur.close()
    conn.close()

# query = """ select current_version(); """
# query_results = query_to_snowflake(query)
# print(query_results)


In [ ]:
put_command = "PUT file:////content/drive/MyDrive/airflow_sendor_data_drive/travel_history.csv @TRAVEL_HISTORY_NAMED_STAGE AUTO_COMPRESS=TRUE"
query_to_snowflake(put_command)

[('travel_history.csv',
  'travel_history.csv.gz',
  195,
  0,
  'NONE',
  'GZIP',
  'SKIPPED',
  '')]

In [ ]:
### Fecth user data from API ###

def fecth_api_data(user_id = 1):
  url = f'https://jsonplaceholder.typicode.com/posts?userId={user_id}'
  api_resp = requests.get(url)
  if api_resp.status_code == 200:
    resp_data = api_resp.json()
    api_data_json = json.dumps(resp_data)
    return api_data_json
  else:
    raise ValueError(f'Api failed with response status {api_resp.status_code}')

In [ ]:
fecth_api_data(user_id = 1)

'[{"userId": 1, "id": 1, "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit", "body": "quia et suscipit\\nsuscipit recusandae consequuntur expedita et cum\\nreprehenderit molestiae ut ut quas totam\\nnostrum rerum est autem sunt rem eveniet architecto"}, {"userId": 1, "id": 2, "title": "qui est esse", "body": "est rerum tempore vitae\\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\\nqui aperiam non debitis possimus qui neque nisi nulla"}, {"userId": 1, "id": 3, "title": "ea molestias quasi exercitationem repellat qui ipsa sit aut", "body": "et iusto sed quo iure\\nvoluptatem occaecati omnis eligendi aut ad\\nvoluptatem doloribus vel accusantium quis pariatur\\nmolestiae porro eius odio et labore et velit aut"}, {"userId": 1, "id": 4, "title": "eum et est occaecati", "body": "ullam et saepe reiciendis voluptatem adipisci\\nsit amet autem assumenda provident rerum culpa\

In [ ]:
user_fetch_start_limit = 1
user_fetch_end_limit = 5
for user_id in range(user_fetch_start_limit,user_fetch_end_limit+1):
  api_data_json = fecth_api_data(user_id)
  insert_query = f""" INSERT INTO {database}.{schema}.{table} (JSON_PAYLOAD) SELECT PARSE_JSON(REPLACE('{api_data_json}', '\n', '\\\\n')) """
  query_results = query_to_snowflake(insert_query, mod='insert')
  print(query_results)

insert operation completed sucessfully
insert operation completed sucessfully
insert operation completed sucessfully
insert operation completed sucessfully
insert operation completed sucessfully


In [ ]:
# user_fetch_limit = 5
# insert_query = f""" INSERT INTO {database}.{schema}.{table} (JSON_PAYLOAD) SELECT PARSE_JSON(Column1) FROM VALUES """
# for user_id in range(1,user_fetch_limit+1):
#   api_data_json = fecth_api_data(user_id)
#   if user_fetch_limit == user_id:
#     insert_query += f""" ('{api_data_json}');"""
#   else:
#     insert_query += f""" ('{api_data_json}')), """

# print(insert_query)
# query_results = query_to_snowflake(insert_query, mod='insert')
# print(query_results)

In [ ]:
# CREATE OR REPLACE PROCEDURE USER_POST_ETL()
#   RETURNS STRING
#   LANGUAGE JAVASCRIPT
#   EXECUTE AS CALLER
# AS
# $$

#   // SET DATABASE
#   snowflake.execute({sqlText: `USE DATABASE SAMPLE_DB;`});

#   // SET SCHEMA
#   snowflake.execute({sqlText: `USE SCHEMA SAMPLE_SCHEMA;`});

#   // CREATE TARGET TABLE IF NOT EXISTS  `
#   snowflake.execute({sqlText:
#   `
#   CREATE TABLE IF NOT EXISTS USER_POST_TARGET (
#   USER_ID VARCHAR,
#   POST_ID VARCHAR,
#   POST_MESSAGE VARCHAR,
#   VENDOR_DATE TIMESTAMP_NTZ(9),
#   SOURCE_PAYLOAD VARIANT,
#   CURRENT_FLAG VARCHAR(1),
#   END_DATE DATE
#   );
#   `
#   });

#   // CREATE FLATTERN SOURCE TABLE FROM STREAM
#   snowflake.execute({sqlText:
#   `

# CREATE OR REPLACE TEMPORARY TABLE FLATTEN_USER_POSTS
#     AS
#     (
#        SELECT
#         FLATTENED.value:userId as USER_ID,
#         FLATTENED.value:id as POST_ID,
#         FLATTENED.value:body as POST_MESSAGE,
#         TO_DATE(INSERT_TS) as VENDOR_DATE,
#         FLATTENED.value as SOURCE_PAYLOAD
#     FROM  USER_POSTS_RAW_DATA_STREAM,
#     LATERAL FLATTEN(INPUT => JSON_PAYLOAD) AS FLATTENED
#     ORDER BY USER_ID, POST_ID
#     );
#     `
#    })
#  // UPDATE FOR EXISTING USER's POST CHANGES
#  snowflake.execute({sqlText:
#   `
#   UPDATE USER_POST_TARGET T
#     SET
#       T.END_DATE = CURRENT_TIMESTAMP(),
#       T.CURRENT_FLAG = 'N'
#     FROM
#       FLATTEN_USER_POSTS S
#     JOIN
#       USER_POST_TARGET TT
#     ON
#        TT.USER_ID = S.USER_ID
#     AND
#        TT.CURRENT_FLAG = 'Y'
#   ;
#  `
#  })

# //INSER FOR EXISTING USER'S POST CHANGES
# snowflake.execute({sqlText:
# `
#   INSERT INTO USER_POST_TARGET
# (
# SELECT
#   S.USER_ID,
#   S.POST_ID,
#   S.POST_MESSAGE,
#   S.VENDOR_DATE,
#   S.SOURCE_PAYLOAD,
#   'Y' AS CURRENT_FLAG,
#   '9999-12-12' AS END_DATE,
# FROM
# FLATTEN_USER_POSTS S
# JOIN
# USER_POST_TARGET T
# ON
#     S.USER_ID = T.USER_ID
# WHERE
#     S.POST_ID  = T.POST_ID
# AND
#     S.POST_MESSAGE <> T.POST_MESSAGE
# );
# `
# })

# // INSERT FOR BRAND NEW USER AND POSTS
# snowflake.execute({sqlText:
# `
# INSERT INTO USER_POST_TARGET
# (
# SELECT
#  S.USER_ID,
#  S.POST_ID,
#  S.POST_MESSAGE,
#  S.VENDOR_DATE,
#  S.SOURCE_PAYLOAD,
#  LEAD('N', 1,'Y') OVER(PARTITION BY S.USER_ID ORDER BY S.USER_ID, S.POST_ID) AS CURRENT_FLAG,
#  LEAD(S.VENDOR_DATE, 1,'9999-12-12') OVER(PARTITION BY S.USER_ID ORDER BY S.USER_ID, S.POST_ID) AS END_DATE
# FROM USER_POST_TARGET T RIGHT JOIN FLATTEN_USER_POSTS S
# ON
#     T.USER_ID = S.USER_ID
# AND
#      T.POST_ID = S.POST_ID
# WHERE
#     T.USER_ID IS NULL
# ORDER BY  S.USER_ID, S.POST_ID
# )
# ;
# `})


# // Define your JavaScript logic here
# var result = "ETL Operation got completed successfully!";
# return result;

# $$;

# CALL USER_POST_ETL();


# CREATE OR REPLACE TASK USER_POST_ETL_TASK
#   WAREHOUSE = COMPUTE_WH
#   SCHEDULE = '5 MINUTE'
# WHEN
#   SYSTEM$STREAM_HAS_DATA('SAMPLE_DB.SAMPLE_SCHEMA.USER_POSTS_RAW_DATA_STREAM')
# AS
# CALL USER_POST_ETL();

# -- START THE TASK
# ALTER TASK USER_POST_ETL_TASK RESUME;

# -- STOP THE TASK
# ALTER TASK USER_POST_ETL_TASK SUSPEND;

# SELECT * FROM SAMPLE_DB.INFORMATION_SCHEMA.TASK_HISTORY
# WHERE TASK_NAME = 'USER_POST_ETL_TASK'
# ORDER BY SCHEDULED_TIME DESC;

# -- TASK HISTORY
# SELECT STATE, * FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY(TASK_NAME=>'USER_POST_ETL_TASK'));

# SELECT * FROM SAMPLE_DB.SAMPLE_SCHEMA.USER_POSTS_RAW_DATA_STREAM;

# SELECT * FROM SAMPLE_DB.SAMPLE_SCHEMA.USER_POST_TARGET ORDER BY END_DATE DESC;

# -- TRUNCATE TABLE SAMPLE_DB.SAMPLE_SCHEMA.USER_POST_TARGET;